In [0]:
from delta.tables import DeltaTable

In [0]:
silver_path = "/Volumes/practice/practice/processed_files/silver"
gold_path = "/Volumes/practice/practice/processed_files/gold"
silver_table = DeltaTable.forPath(spark, silver_path)
df_silver = silver_table.toDF()


In [0]:
from pyspark.sql.functions import sum

df_gold_new = df_silver.groupBy(
    "product", 
    "region"
).agg(
    sum("amount").alias("Revenue"),

)

In [0]:
try:
    gold_table = DeltaTable.forPath(spark, gold_path)
    
    gold_table.alias("old").merge(
        df_gold_new.alias("new"),
        "old.product = new.product AND old.region = new.region"
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()

except:
    df_gold_new.write.format("delta").mode("overwrite").save(gold_path)